# Business case

* Predict the outcome of a baseball game based on match metrics (other than the score)
* Helping the team focus on the right skills to train for as well as chosing the right game strategy

* create 'historical' model then deploy it for a team specifically


## Further work

* Deploy and fine tune model to a specific team
* Be able to identify the players that "match the model"


In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv('MLB-GameLogs-1871_2016.csv', low_memory = False)
df

date  number_of_game day_of_week v_name v_league  v_game_number  \
0      19460416               0         Tue    SLA       AL              1   
1      19460416               0         Tue    NYA       AL              1   
2      19460416               0         Tue    BOS       AL              1   
3      19460416               0         Tue    CLE       AL              1   
4      19460416               0         Tue    BRO       NL              1   
...         ...             ...         ...    ...      ...            ...   
94927  20161002               0         Sun    MIL       NL            162   
94928  20161002               0         Sun    NYN       NL            162   
94929  20161002               0         Sun    LAN       NL            162   
94930  20161002               0         Sun    PIT       NL            162   
94931  20161002               0         Sun    MIA       NL            161   

      h_name h_league  h_game_number  v_score  ...  h_player_7_name  \
0        DET       AL              1        1  ...    Pinky Higgins   
1        PHA       AL              1        5  ...      George Kell   
2        WS1       AL              1        6  ...     Jerry Priddy   
3        CHA       AL              1        1  ...   Thurman Tucker   
4        BSN       NL              1        3  ...   Skippy Roberge   
...      ...      ...            ...      ...  ...              ...   
94927    COL       NL            162        6  ...      Pat Valaika   
94928    PHI       NL            162        2  ...    Andres Blanco   
94929    SFN       NL            162        1  ...        Joe Panik   
94930    SLN       NL            162        4  ...      Jedd Gyorko   
94931    WAS       NL            162        7  ...   Danny Espinosa   

       h_player_7_def_pos h_player_8_id   h_player_8_name h_player_8_def_pos  \
0                     5.0      richp101     Paul Richards                2.0   
1                     5.0      handg101      Gene Handley                4.0   
2                     4.0      evana101          Al Evans                2.0   
3                     8.0      tresm101        Mike Tresh                2.0   
4                     5.0      culld101       Dick Culler                6.0   
...                   ...           ...               ...                ...   
94927                 4.0      adamc001  Cristhian Adames                6.0   
94928                 6.0      altha001     Aaron Altherr                9.0   
94929                 4.0      gillc001   Conor Gillaspie                5.0   
94930                 4.0      gricr001    Randal Grichuk                8.0   
94931                 6.0      lobaj001      Jose Lobaton                2.0   

      h_player_9_id   h_player_9_name  h_player_9_def_pos  additional_info  \
0          newhh101     Hal Newhouser                 1.0              NaN   
1          chrir101  Russ Christopher                 1.0              NaN   
2          wolfr102       Roger Wolff                 1.0              NaN   
3          dietb101     Bill Dietrich                 1.0              NaN   
4          sainj101       Johnny Sain                 1.0              NaN   
...             ...               ...                 ...              ...   
94927      marqg001    German Marquez                 1.0              NaN   
94928      eickj001    Jerad Eickhoff                 1.0              NaN   
94929      moorm003        Matt Moore                 1.0              NaN   
94930      waina001   Adam Wainwright                 1.0              NaN   
94931      schem001      Max Scherzer                 1.0              NaN   

      acquisition_info  
0                    Y  
1                    Y  
2                    Y  
3                    Y  
4                    Y  
...                ...  
94927                Y  
94928                Y  
94929                Y  
94930                Y  
94931                Y  

[94932 rows x 161 colu

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

# Data Cleaning & Exploration

Let's look at the teams:

In [4]:
# Dropping redundant columns
dropping = []
for column in df.columns[7:]:
    if column.endswith('_def_pos'):
        dropping.append(column)
    elif '_name' in column:
        dropping.append(column)
    elif '_info' in column:
        dropping.append(column)
    elif 'lf_umpire_id' in column:
        dropping.append(column)
    elif 'rf_umpire_id' in column:
        dropping.append(column)
df = df.drop(columns = dropping)

# Removing the 2 forfeited games & column
df = df[df['forefeit'].isna()==True]
df = df.drop(columns = 'forefeit')

# Removing interrupted games & completion column
df = df[df['completion'].isna()==True]
df = df.drop(columns = 'completion')

# Remove protested games & column
df = df[df['protest'].isna()==True]
df = df.drop(columns = 'protest')

# Remove multi-headed games
df = df[df['number_of_game'] == 0]
df = df.drop(columns = 'number_of_game')

# Remove draws
df = df[df['h_score'] != df['v_score']]

In [5]:
# Random Weighted Imputing for day_night column
choice = sorted(df[df.day_night.isna() == False]['day_night'].unique())
prob = df[df.day_night != 0]['day_night'].value_counts(normalize=True).sort_index()
df['day_night'] = df.day_night.map(lambda x: np.random.choice(choice, 1, p = prob)[0] if x not in choice else x)

# Median Imputing for attendance & length_minutes missing values
df['attendance'] = df['attendance'].fillna(value= df['attendance'].median)
df['length_minutes'] = df['length_minutes'].fillna(value= df['length_minutes'].median)

In [6]:
# Reset Index & Assign Match ID
df.reset_index(inplace = True)
df.rename(columns = {'index': 'match_id'}, inplace = True)

In [7]:
# Check for missing values
for column in df.columns:
    if df[column].isna().sum() != 0:
        print (f"Missing in {column}: {df[column].isna().sum()}")

Missing in 2b_umpire_id: 4872
Missing in 3b_umpire_id: 128
Missing in saving_pitcher_id: 46722
Missing in winning_rbi_batter_id: 5519


In [8]:
# Categorize winning team for restructuring dataset
winning_team = []
for i in range(0,len(df)):
    if df['h_score'][i] > df['v_score'][i] == True:
        winning_team.append('Home')
    else:
        winning_team.append('Visitor')
df['winning_team'] = winning_team

# Define unique home/visitor stats
h_stats = []
for column in df.columns:
    if str(column).startswith('h_'):
        h_stats.append(column)

v_stats = []
for column in df.columns:
    if str(column).startswith('v_'):
        v_stats.append(column)

In [10]:
# Extracting only winning team and respective statistics
dfwin1 = df[df['winning_team'] == 'Home']
dfwin1 = dfwin1.drop(columns = v_stats, axis = 1)

dfwin2 = df[df['winning_team'] == 'Visitor']
dfwin2 = dfwin2.drop(columns = h_stats, axis = 1)

# Renaming h_ and v_ columns
for column in dfwin1.columns:
    if str(column).startswith('h_') == True:
        dfwin1.rename(columns = {column: column[2:]}, inplace = True)
        
for column in dfwin2.columns:
    if str(column).startswith('v_') == True:
        dfwin2.rename(columns = {column: column[2:]}, inplace = True)

# Merging both win df
dfwin = pd.concat([dfwin1, dfwin2], sort = False) 

# Assigning boolean target variable "winner"
dfwin["winner"] = np.nan
dfwin['winner'].fillna(value = 1, inplace = True)

In [14]:
# Extracting only losing team and respective statistics
dfloss1 = df[df['winning_team'] == 'Home']
dfloss1 = dfloss1.drop(columns = h_stats, axis = 1)

dfloss2 = df[df['winning_team'] == 'Visitor']
dfloss2 = dfloss2.drop(columns = v_stats, axis = 1)

# Renaming h_ and v_ columns
for column in dfloss1.columns:
    if str(column).startswith('v_') == True:
        dfloss1.rename(columns = {column: column[2:]}, inplace = True)
        
for column in dfloss2.columns:
    if str(column).startswith('h_') == True:
        dfloss2.rename(columns = {column: column[2:]}, inplace = True)

# Merging both loss df
dfloss = pd.concat([dfloss1, dfloss2], sort = False)

# Assigning boolean target variable "winner"
dfloss["winner"] = np.nan
dfloss['winner'].fillna(value = 0, inplace = True)

In [21]:
# Combine both win and loss dataframe
df = pd.concat([dfwin, dfloss], sort = False)
df